In [ ]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 

In [ ]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df[:5]

EDA 

In [ ]:
df.columns
df.info()  # data types
df.describe()  # summary statistics


In [ ]:
df['Churn'].value_counts(normalize=True)
df['Churn'].value_counts().plot(kind='bar')

In [ ]:
import seaborn as sns
sns.boxplot(x='Churn', y='MonthlyCharges', data=df)

In [ ]:
df.shape

In [ ]:
df.dtypes

Dataset Preprocessing

In [ ]:
df.drop('customerID', axis=1, inplace=True)
df[:3]

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [ ]:
df.dtypes

In [ ]:
def checkUnique():
    for column in df.columns:
        print(f"{column} : {df[column].unique()}")

In [ ]:
df['MultipleLines'].replace('No phone service', 'No', inplace=True)
df['OnlineSecurity'].replace('No internet service', 'No', inplace=True)
df['OnlineBackup'].replace('No internet service', 'No', inplace=True)

df['DeviceProtection'].replace('No internet service', 'No', inplace=True)
df['TechSupport'].replace('No internet service', 'No', inplace=True)
df['StreamingTV'].replace('No internet service', 'No', inplace=True)
df['StreamingMovies'].replace('No internet service', 'No', inplace=True)

In [ ]:
checkUnique()

In [ ]:
object_columns = df.select_dtypes(include='object')
for col in object_columns.columns:
    df[col].replace('Yes',1, inplace=True)
    df[col].replace('No',0, inplace=True)

df['gender'].replace('Male',0, inplace=True) 
df['gender'].replace('Female',1, inplace=True) 

In [ ]:
checkUnique()

In [ ]:
df.dtypes

In [ ]:
object_columns = df.select_dtypes(include='object')

df_encoded = pd.get_dummies(df, columns=object_columns.columns)

df = pd.get_dummies(df, columns=object_columns.columns)


In [ ]:
df.dtypes

In [ ]:
df[:3]

In [ ]:
checkUnique()

In [ ]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges'] 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])


In [ ]:
df = df.dropna()

In [ ]:
! pip install seaborn

In [ ]:
X = df.drop('Churn',axis='columns')
y = df['Churn']

from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
x_train.shape

In [ ]:
y_test.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras

def get_model(x_train,y_train,epochs):
    model = keras.Sequential([
        keras.layers.Dense(26, input_shape=(26,), activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

    model.fit(x_train, y_train, epochs)
    return model 

In [ ]:
from sklearn.metrics import classification_report
def model_evaluate(model,x_test,y_test):
    model.evaluate(x_test,y_test)
    pred = model.predict(x_test)
    i = 0 
    y_pred = []
    for p in pred:

        if p >= 0.5:
            y_pred.append(1)
            # print(f"Actual : {y_test.iloc[i]} | Predicited : 1")
        
        else: 
            y_pred.append(0)
            # print(f"Actual : {y_test.iloc[i]} | Predicited : 0")
        
        i += 1 

    print(classification_report(y_test,y_pred))
   

In [ ]:
model = get_model(x_train,y_train,10) #model training 
model_evaluate(model,x_test,y_test)   #model evaluation 

Handling Imbalanced Churn Value Count by Over Sampling

In [ ]:
zero_count, one_count = df.Churn.value_counts()
zero_count, one_count

In [ ]:
one_rows = df[df["Churn"]==1]
zero_rows= df[df["Churn"]==0]


In [ ]:
one_rows = one_rows.sample(zero_count,replace=True, random_state=3)
one_rows[:3]

In [ ]:
one_rows.shape

In [ ]:
zero_rows.shape

In [ ]:
df_balanced = pd.concat([zero_rows,one_rows])
df_balanced.shape

In [ ]:
x = df_balanced.drop('Churn',axis=1)
y = df_balanced["Churn"]

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=34, stratify=y)

In [ ]:
y_train.value_counts()

In [ ]:
x_train.shape

In [ ]:
model = get_model(x_train,y_train,10)
model_evaluate(model,x_test,y_test)

Under Sampling

In [ ]:
one_rows = df[df["Churn"]==1]
zero_rows= df[df["Churn"]==0] 

zero_rows = zero_rows.sample(one_count)
zero_rows.shape

In [ ]:
df_balanced_under = pd.concat([one_rows,zero_rows])
df_balanced_under["Churn"].value_counts()

In [ ]:
x = df_balanced_under.drop('Churn',axis=1)
y = df_balanced_under["Churn"]

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=34, stratify=y)

In [ ]:
model = get_model(x_train,y_train,10)
model_evaluate(model,x_test,y_test)